In [8]:
#importing required packages 
import datetime
from datetime import timedelta
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytz
import os
import time

In [9]:
#connecting to the api
url = "https://developer-apis.awair.is/v1/orgs/2970"
payload = {}
headers= {'x-api-key':'4iS73nI45Lkt9ydm8i9wb4BADCCKf1Y9'}
orgs = requests.request("GET", url, headers=headers, data = payload)

print(orgs.text.encode('utf8'))

b'{\n "id": "2970",\n "display_name": "UVA",\n "active": true\n}'


In [44]:
#Defining dictionaries for sensor ids and desired variables (keyList) you want to extract
#sensor_ids = {"sensor_name": "sensor_id"}
# sensor_ids = {"281-omni_4":13274}
sensor_ids = {"211_TV_1":26820}
keyList = ["pm25", "co2", "voc", "humid", "temp", "pm10_est", "score", "lux", "spl_a"]
# keyList = ["pm25", "co2", "voc", "humid", "temp", "pm10_est", "score", "lux", "spl_a"]

In [46]:
#Define the desired time zone
eastern = pytz.timezone('US/Eastern')


#Define the desired range of time you want to extract the data from
firstDate = datetime.datetime(2022,6,15,0,0,0)
lastDate  = datetime.datetime(2022,11,8,0,0,0)
hourSteps  = 1

date = firstDate

In [55]:
for sensors, ids in sensor_ids.items():
    datadict = {}
    for i in keyList:
        datadict[i] = []
    
    while date <= lastDate:

        t = date        
        x = t + timedelta(hours = 1)
        
        t = t.isoformat()
        x = x.isoformat()
        url = "https://developer-apis.awair.is/v1/orgs/2970/devices/awair-omni/{}/air-data/raw?from={}&to={}&limit=360&desc=false&fahrenheit=false".format(ids,t, x)
        # url = "https://developer-apis.awair.is/v1/orgs/2970/devices/awair-omni/{}/air-data/5-min-avg?from={}&to={}&limit=100&desc=false&fahrenheit=false".format(ids,t, x)
 

        devices = requests.request("GET", url, headers=headers, data = payload)
        a = devices.text.encode('utf8')
        
        a = json.loads(a)
        
        try:
            a["data"]
        except:
            print(date)

            current_time = datetime.datetime.now().strftime("%H:%M:%S")
            print("Current Time =", current_time)

            time.sleep(65)
            devices = requests.request("GET", url, headers=headers, data = payload)
            a = devices.text.encode('utf8')
            a = json.loads(a)
            # break

        # print(a)
        for i in range(len(a["data"])):
            for j in range(len(a["data"][i]["sensors"])):                        
                if a["data"][i]["sensors"][j]['comp'] in datadict.keys():
#                         print(a['data'][i]['timestamp'])
                    datadict[a["data"][i]["sensors"][j]['comp']].append([a['data'][i]['timestamp'], a["data"][i]["sensors"][j]['value']])
                               
        date += datetime.timedelta(hours=hourSteps)                  
                        
    
    for ikey, key in zip(range(len(datadict.keys())), datadict.keys()):
        column_names_key = ["timestamp", key]
        df_key = pd.DataFrame(datadict[key], columns = column_names_key)
        
        if ikey == 0:
            df_all_key = df_key.copy()
        
        elif ikey > 0:
            df_all_key = pd.merge(df_all_key, df_key, on='timestamp')
    
    
    final_file = df_all_key.reset_index()
    final_file = final_file.set_index(pd.DatetimeIndex(final_file['timestamp']))
    final_file = final_file.drop(['index'], axis=1)
    final_file = final_file.drop(['timestamp'], axis=1)
    final_file = final_file[~final_file.index.duplicated()] 
    # final_file.index = final_file.index.tz_localize(pytz.utc).tz_convert(eastern)
    final_file.index = final_file.index.tz_convert(eastern)
    final_file.to_csv('data_sensor_{}.csv'.format(sensors))
    print("file is saved")

file is saved


In [43]:
# import requests

# url = "https://developer-apis.awair.is/v1/orgs/2970/devices"

# payload={}
# headers = {
#   'x-api-key': '4iS73nI45Lkt9ydm8i9wb4BADCCKf1Y9'
# }

# response = requests.request("GET", url, headers=headers, data=payload)

#print(response.text)

In [42]:
# import requests

# url = "https://developer-apis.awair.is/v1/orgs/2970/devices/awair-omni/13274/air-data/raw"
# payload={}
# headers = {
#   'x-api-key': '4iS73nI45Lkt9ydm8i9wb4BADCCKf1Y9'
# }

# response = requests.request("GET", url, headers=headers, data=payload)

#print(response.text)